# Daily Email Summary System

This notebook reads all emails from today, generates a summary using Ollama (local LLM), and sends the summary as a new email.

## Setup Instructions

1. Install dependencies: `pip install -r requirements.txt`
2. Make sure Ollama is installed and running: https://ollama.ai
3. Pull a model: `ollama pull llama3` (or another model)
4. Configure your email credentials in the configuration cell below


In [20]:
# Import required libraries
import imaplib
import smtplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta
import ollama
import os
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")


Libraries imported successfully!


In [21]:
## Configuration

Set up your email credentials and Ollama model settings here.


SyntaxError: invalid syntax (518248051.py, line 3)

In [ ]:
# Email Configuration
# For Gmail, you'll need to use an "App Password" instead of your regular password
# Go to: Google Account > Security > 2-Step Verification > App passwords

EMAIL_ADDRESS = os.getenv('EMAIL_ADDRESS', 'yeshwanthreddy.bunny@gmail.com')
EMAIL_PASSWORD = os.getenv('EMAIL_PASSWORD', '')  # Set in .env file or replace here
IMAP_SERVER = os.getenv('IMAP_SERVER', 'imap.gmail.com')
IMAP_PORT = 993
SMTP_SERVER = os.getenv('SMTP_SERVER', 'smtp.gmail.com')
SMTP_PORT = 587

# Ollama Configuration
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'llama3')  # Change to your preferred model

# Summary Email Configuration
SUMMARY_RECIPIENT = os.getenv('SUMMARY_RECIPIENT', EMAIL_ADDRESS)  # Where to send the summary

print("Configuration loaded!")
print(f"Email: {EMAIL_ADDRESS}")
if not EMAIL_PASSWORD or EMAIL_PASSWORD == '':
    print("⚠️  WARNING: Email password not set. You need to:")
    print("   1. Get a Gmail App Password (see instructions below)")
    print("   2. Add it to .env file or set EMAIL_PASSWORD here")
else:
    print("✅ Email password configured")


Configuration loaded!
Email: yeshwanthreddy.bunny@gmail.com
✅ Email password configured


## Email Reading Functions


In [ ]:
def decode_mime_words(s):
    """Decode MIME encoded words in email headers"""
    return email.header.decode_header(s)

def get_email_body(msg):
    """Extract the body text from an email message"""
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))
            
            if content_type == "text/plain" and "attachment" not in content_disposition:
                try:
                    payload = part.get_payload(decode=True)
                    if payload:
                        charset = part.get_content_charset() or 'utf-8'
                        body += payload.decode(charset, errors='ignore')
                except Exception as e:
                    print(f"Error decoding part: {e}")
    else:
        try:
            payload = msg.get_payload(decode=True)
            if payload:
                charset = msg.get_content_charset() or 'utf-8'
                body = payload.decode(charset, errors='ignore')
        except Exception as e:
            print(f"Error decoding body: {e}")
    
    return body.strip()

def get_today_emails(email_address, password, imap_server, imap_port):
    """Fetch all emails received today"""
    try:
        # Connect to IMAP server
        mail = imaplib.IMAP4_SSL(imap_server, imap_port)
        mail.login(email_address, password)
        mail.select('inbox')
        
        # Get today's date in the format required by IMAP
        today = datetime.now().date()
        date_str = today.strftime("%d-%b-%Y")
        
        # Search for emails from today
        status, messages = mail.search(None, f'(SINCE {date_str})')
        email_ids = messages[0].split()
        
        emails = []
        
        for email_id in email_ids:
            try:
                # Fetch the email
                status, msg_data = mail.fetch(email_id, '(RFC822)')
                email_body = msg_data[0][1]
                msg = email.message_from_bytes(email_body)
                
                # Extract email information
                subject = decode_mime_words(msg['Subject'])[0][0]
                if isinstance(subject, bytes):
                    subject = subject.decode('utf-8', errors='ignore')
                
                from_addr = decode_mime_words(msg['From'])[0][0]
                if isinstance(from_addr, bytes):
                    from_addr = from_addr.decode('utf-8', errors='ignore')
                
                date = msg['Date']
                body = get_email_body(msg)
                
                emails.append({
                    'subject': subject,
                    'from': from_addr,
                    'date': date,
                    'body': body
                })
            except Exception as e:
                print(f"Error processing email {email_id}: {e}")
                continue
        
        mail.close()
        mail.logout()
        
        return emails
    
    except Exception as e:
        print(f"Error connecting to email server: {e}")
        return []

print("Email reading functions defined!")


Email reading functions defined!


## Ollama Summarization Function


In [ ]:
def summarize_emails_with_ollama(emails, model='llama3'):
    """Generate a summary of all emails using Ollama"""
    if not emails:
        return "No emails received today."
    
    # Prepare email content for summarization
    email_text = ""
    for i, email_data in enumerate(emails, 1):
        email_text += f"\n--- Email {i} ---\n"
        email_text += f"From: {email_data['from']}\n"
        email_text += f"Subject: {email_data['subject']}\n"
        email_text += f"Date: {email_data['date']}\n"
        # Truncate body if too long (to avoid token limits)
        body = email_data['body'][:2000] if len(email_data['body']) > 2000 else email_data['body']
        email_text += f"Body: {body}\n"
    
    # Create prompt for summarization
    prompt = f"""Please provide a concise daily email summary. Organize it by:
1. Total number of emails
2. Key topics/themes
3. Important emails (with sender and brief description)
4. Action items if any

Here are today's emails:
{email_text}

Please provide a well-formatted summary:"""

    try:
        # Call Ollama API
        response = ollama.chat(model=model, messages=[
            {
                'role': 'system',
                'content': 'You are a helpful assistant that summarizes emails in a clear and organized manner.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        summary = response['message']['content']
        return summary
    
    except Exception as e:
        print(f"Error calling Ollama: {e}")
        print("Make sure Ollama is running and the model is available.")
        return f"Error generating summary: {e}"

print("Ollama summarization function defined!")


Ollama summarization function defined!


## Email Sending Function


In [ ]:
def send_summary_email(sender_email, sender_password, recipient_email, summary, smtp_server, smtp_port):
    """Send the email summary"""
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = f"Daily Email Summary - {datetime.now().strftime('%Y-%m-%d')}"
        
        # Add body to email
        body = f"""
Daily Email Summary
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

{summary}

---
This is an automated summary generated using Ollama.
"""
        msg.attach(MIMEText(body, 'plain'))
        
        # Send email
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, recipient_email, text)
        server.quit()
        
        print(f"Summary email sent successfully to {recipient_email}")
        return True
    
    except Exception as e:
        print(f"Error sending email: {e}")
        return False

print("Email sending function defined!")


Email sending function defined!


## Main Workflow

Run this cell to execute the complete workflow: fetch emails, generate summary, and send it.


In [ ]:
def daily_email_summary_workflow():
    """Main workflow: Read emails, summarize, and send summary"""
    print("=" * 60)
    print("Starting Daily Email Summary Workflow")
    print("=" * 60)
    
    # Step 1: Fetch today's emails
    print("\n[1/3] Fetching today's emails...")
    emails = get_today_emails(EMAIL_ADDRESS, EMAIL_PASSWORD, IMAP_SERVER, IMAP_PORT)
    print(f"Found {len(emails)} email(s) today")
    
    if not emails:
        print("No emails found for today. Exiting.")
        return
    
    # Step 2: Generate summary using Ollama
    print(f"\n[2/3] Generating summary using Ollama ({OLLAMA_MODEL})...")
    summary = summarize_emails_with_ollama(emails, OLLAMA_MODEL)
    print("Summary generated successfully!")
    print("\n" + "=" * 60)
    print("SUMMARY PREVIEW:")
    print("=" * 60)
    print(summary)
    print("=" * 60)
    
    # Step 3: Send summary email
    print(f"\n[3/3] Sending summary email to {SUMMARY_RECIPIENT}...")
    success = send_summary_email(
        EMAIL_ADDRESS, 
        EMAIL_PASSWORD, 
        SUMMARY_RECIPIENT, 
        summary, 
        SMTP_SERVER, 
        SMTP_PORT
    )
    
    if success:
        print("\n✅ Workflow completed successfully!")
    else:
        print("\n❌ Workflow completed with errors. Check the output above.")

# Run the workflow
# Uncomment the line below to execute
# daily_email_summary_workflow()


## Test Individual Components

You can test each component separately before running the full workflow.


In [ ]:
# Test: Fetch emails (without sending summary)
test_emails = get_today_emails(EMAIL_ADDRESS, EMAIL_PASSWORD, IMAP_SERVER, IMAP_PORT)
print(f"Found {len(test_emails)} emails")
for i, email_data in enumerate(test_emails[:3], 1):  # Show first 3
    print(f"\nEmail {i}:")
    print(f"  From: {email_data['from']}")
    print(f"  Subject: {email_data['subject']}")
    print(f"  Body preview: {email_data['body'][:200]}...")


Found 35 emails

Email 1:
  From: Elearn Support <support-elearn@nptel.iitm.ac.in>
  Subject: Hurry!!! Enhance Your Skills with NPTEL+ Certified Workshop - NEP 3
  Body preview: <p>Dear Candidate,</p>
<p><span style="font-size: small;"><br /></span><span style="font-family: Arial; font-size: small;">We are delighted to share an exciting opportunity to attend an NPTEL+ worksh...

Email 2:
  From: Elearn Support <support-elearn@nptel.iitm.ac.in>
  Subject: Hurry!!! Enhance Your Skills with NPTEL+ Certified Workshop - NEP 3
  Body preview: <p>Dear Candidate,</p>
<p><span style="font-size: small;"><br /></span><span style="font-family: Arial; font-size: small;">We are delighted to share an exciting opportunity to attend an NPTEL+ worksh...

Email 3:
  From: "Kohl's Friends & Family" <kohls@s.kohls.com>
  Subject: IT'S HERE! Extra 25% off for our friends & fam 🎉
  Body preview: https://click.s.kohls.com/?qs=78ee3607d96da6863eba8e8b87ccdbd4b9066bda7a99194887160a3844486106c79dfa4c252d69f5f4b3

In [ ]:
# Test: Check Ollama connection
try:
    response = ollama.chat(model=OLLAMA_MODEL, messages=[
        {'role': 'user', 'content': 'Say hello in one sentence.'}
    ])
    print("✅ Ollama is working!")
    print(f"Response: {response['message']['content']}")
except Exception as e:
    print(f"❌ Ollama error: {e}")
    print("Make sure Ollama is running: ollama serve")
    print(f"Make sure model is available: ollama pull {OLLAMA_MODEL}")


✅ Ollama is working!
Response: Hello!


## Test Email Sending

Test if email sending works correctly.


In [22]:
# Test email sending directly
test_summary = "This is a test email to verify the email sending functionality works."
print(f"Testing email send to: {SUMMARY_RECIPIENT}")
print(f"From: {EMAIL_ADDRESS}")

result = send_summary_email(
    EMAIL_ADDRESS,
    EMAIL_PASSWORD,
    SUMMARY_RECIPIENT,
    test_summary,
    SMTP_SERVER,
    SMTP_PORT
)

if result:
    print("\n✅ Test email sent! Check your inbox (and spam folder).")
    print(f"   Look for subject: 'Daily Email Summary - {datetime.now().strftime('%Y-%m-%d')}'")
else:
    print("\n❌ Failed to send test email. Check the error above.")


Testing email send to: yeshwanthreddy.bunny@gmail.com
From: yeshwanthreddy.bunny@gmail.com
Summary email sent successfully to yeshwanthreddy.bunny@gmail.com

✅ Test email sent! Check your inbox (and spam folder).
   Look for subject: 'Daily Email Summary - 2025-12-15'
